In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/OHLCV/Time-Series-Library
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/OHLCV/Time-Series-Library
btc_1min_embeddings.pkl       requirements.txt
checkpoints		      result_long_term_forecast.txt
CONTRIBUTING.md		      results
data_provider		      run.py
dataset			      scripts
Embedding_extractor.ipynb     test_btc_embeddings.pkl
embedding_to_csv_mapping.npy  test_embed_ohlcv.csv
exp			      test_results
fixed_data_btc_1min1.csv      timesnet_embeddings_768.npy
fixed_data_btc_1min2.csv      timesnet_embeddings.pkl
layers			      tsl
LICENSE			      TSLib
models			      tutorial
pic			      utils
README.md


In [ ]:
!pip install reformer_pytorch
!pip install sktime

In [ ]:
import torch
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast
import argparse
import pickle

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Final_Block/test_embed_ohlcv.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15978 entries, 0 to 15977
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        15978 non-null  object 
 1   open        15978 non-null  float64
 2   high        15978 non-null  float64
 3   low         15978 non-null  float64
 4   close       15978 non-null  float64
 5   volume      15978 non-null  float64
 6   pre_return  15978 non-null  float64
 7   return      15978 non-null  float64
dtypes: float64(7), object(1)
memory usage: 998.8+ KB


In [ ]:
import pickle
import argparse

In [ ]:
import torch
import numpy as np
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast

# Extract backbone embeddings
def extract_timesnet_embeddings(checkpoint_path, data_config):
    class Args:
        def __init__(self, **kwargs):
            for k, v in kwargs.items():
                setattr(self, k, v)

    args = Args(**data_config)

    exp = Exp_Long_Term_Forecast(args)

    if checkpoint_path and checkpoint_path != 'auto':
        exp.model.load_state_dict(torch.load(checkpoint_path))
        print(f"Model loaded from {checkpoint_path}")

    exp.model.eval()

    train_data, train_loader = exp._get_data(flag='test')

    backbone_embeddings = []

    def backbone_hook(module, input, output):
        """after TimesBlock processing"""
        if isinstance(output, torch.Tensor):
            backbone_embeddings.append(output.detach().cpu().clone())
            print(f"Captured backbone embeddings: {output.shape}")

    timesblock_handle = None
    for name, module in exp.model.named_modules():
        if name == 'model.0':
            timesblock_handle = module.register_forward_hook(backbone_hook)
            print(f"Registered hook on TimesBlock: {name}")
            break

    if not timesblock_handle:
        print("TimesBlock not found, trying alternative approach...")
        return extract_with_forward_modification(exp)

    all_embeddings = []
    all_labels = []

    print(f"Processing {len(test_loader)} batches...")

    with torch.no_grad():
        for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
            batch_x = batch_x.float().to(exp.device)
            batch_y = batch_y.float().to(exp.device)
            batch_x_mark = batch_x_mark.float().to(exp.device)
            batch_y_mark = batch_y_mark.float().to(exp.device)

            backbone_embeddings.clear()

            output = exp.model(batch_x, batch_x_mark, batch_y, batch_y_mark)

            if backbone_embeddings:
                embeddings = backbone_embeddings[0]
                all_embeddings.append(embeddings)
                all_labels.append(batch_y[:, -1:, :].cpu())

                if i == 0:
                    print(f"Backbone embeddings shape: {embeddings.shape}")
                    print(f"Model output shape: {output.shape}")

            if i % 20 == 0:
                print(f"Processed {i}/{len(test_loader)} batches")

    if timesblock_handle:
        timesblock_handle.remove()

    if all_embeddings:
        final_embeddings = torch.cat(all_embeddings, dim=0).numpy()
        final_labels = torch.cat(all_labels, dim=0).numpy()

        print(f"\n=== Extraction Complete ===")
        print(f"Final embeddings shape: {final_embeddings.shape}")
        print(f"Final labels shape: {final_labels.shape}")

        np.save('timesnet_backbone_embeddings.npy', final_embeddings)
        np.save('timesnet_labels.npy', final_labels)
        print("Embeddings saved to timesnet_backbone_embeddings.npy")
        print("Labels saved to timesnet_labels.npy")

        return final_embeddings, final_labels
    else:
        print("No embeddings captured!")
        return None, None


config = {
    'task_name': 'classification',
    'is_training': 0,
    'model': 'TimesNet',
    'data': 'custom',
    'root_path': '/content/drive/MyDrive/Final_Block/',
    'data_path': 'test_embed_ohlcv.csv',
    'features': 'MS',
    'target': 'return',
    'freq': 'h',
    'seasonal_patterns': 'Monthly',
    'inverse': 0,
    'seq_len': 48,
    'label_len': 24,
    'pred_len': 1,
    'e_layers': 1,
    'd_layers': 1,
    'factor': 3,
    'num_kernels': 6,
    'enc_in': 7,
    'dec_in': 7,
    'c_out': 1,
    'd_model': 768,
    'd_ff': 32,
    'top_k': 5,
    'des': 'QuickTest',
    'itr': 1,
    'batch_size': 32,
    'dropout': 0.1,
    'embed': 'timeF',
    'activation': 'gelu',
    'num_workers': 10,
    'patience': 3,
    'learning_rate': 0.0001,
    'use_gpu': True,
    'gpu_type': 'cuda',
    'use_multi_gpu': False,
    'gpu': 0,
    'data_split': '[0.0, 0.0, 1.0]',
    'num_class': 2
}


checkpoint_path = "./checkpoints/long_term_forecast_test_btc_ohlcv_timesnet_TimesNet_custom_ftMS_sl48_ll24_pl1_dm64_nh8_el1_dl1_df32_expand2_dc4_fc3_ebtimeF_dtTrue_QuickTest_0/checkpoint.pth"

embeddings, labels = extract_timesnet_embeddings(checkpoint_path, config)

Use GPU: cuda:0


RuntimeError: Error(s) in loading state_dict for Model:
	Unexpected key(s) in state_dict: "predict_linear.weight", "predict_linear.bias". 
	size mismatch for model.0.conv.0.kernels.0.weight: copying a param with shape torch.Size([32, 64, 1, 1]) from checkpoint, the shape in current model is torch.Size([32, 768, 1, 1]).
	size mismatch for model.0.conv.0.kernels.1.weight: copying a param with shape torch.Size([32, 64, 3, 3]) from checkpoint, the shape in current model is torch.Size([32, 768, 3, 3]).
	size mismatch for model.0.conv.0.kernels.2.weight: copying a param with shape torch.Size([32, 64, 5, 5]) from checkpoint, the shape in current model is torch.Size([32, 768, 5, 5]).
	size mismatch for model.0.conv.0.kernels.3.weight: copying a param with shape torch.Size([32, 64, 7, 7]) from checkpoint, the shape in current model is torch.Size([32, 768, 7, 7]).
	size mismatch for model.0.conv.0.kernels.4.weight: copying a param with shape torch.Size([32, 64, 9, 9]) from checkpoint, the shape 

In [ ]:
import torch
import numpy as np
from exp.exp_long_term_forecasting import Exp_Long_Term_Forecast

def debug_model_structure(checkpoint_path, data_config):
    """
    Debug the TimesNet model structure to find the right layers for embedding extraction
    """

    class Args:
        def __init__(self, **kwargs):
            for k, v in kwargs.items():
                setattr(self, k, v)

    args = Args(**data_config)

    exp = Exp_Long_Term_Forecast(args)

    if checkpoint_path and checkpoint_path != 'auto':
        exp.model.load_state_dict(torch.load(checkpoint_path))

    print("=== TimesNet Model Structure ===")
    for name, module in exp.model.named_modules():
        print(f"{name}: {type(module).__name__}")

    print("\n=== Model Children ===")
    for name, child in exp.model.named_children():
        print(f"{name}: {type(child).__name__}")
        if hasattr(child, '__len__') and not isinstance(child, torch.nn.Parameter):
            try:
                print(f"  Length: {len(child)}")
            except:
                pass

    return exp

def extract_embeddings_with_multiple_hooks(exp, target_layers=None):
    """
    Extract embeddings by hooking into multiple potential layers
    """
    if target_layers is None:
        target_layers = [
            'layer_stack',
            'encoder',
            'enc_layers',
            'backbone',
            'layers',
            'projection'
        ]

    test_data, test_loader = exp._get_data(flag='test')

    layer_embeddings = {}

    def create_hook(layer_name):
        def hook_fn(module, input, output):
            if isinstance(output, torch.Tensor):
                layer_embeddings[layer_name] = output.detach().cpu().clone()
                print(f"Captured from {layer_name}: {output.shape}")
        return hook_fn

    hook_handles = []
    for name, module in exp.model.named_modules():
        for target in target_layers:
            if target in name.lower():
                handle = module.register_forward_hook(create_hook(name))
                hook_handles.append(handle)
                print(f"Registered hook on: {name}")

    if not hook_handles:
        print("No target layers found, hooking all direct children...")
        for name, module in exp.model.named_children():
            handle = module.register_forward_hook(create_hook(name))
            hook_handles.append(handle)
            print(f"Registered hook on child: {name}")

    exp.model.eval()
    all_layer_embeddings = {layer: [] for layer in layer_embeddings.keys()}

    # Extract embeddings from one batch
    with torch.no_grad():
        for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
            if i > 0:
                break

            batch_x = batch_x.float().to(exp.device)
            batch_y = batch_y.float().to(exp.device)
            batch_x_mark = batch_x_mark.float().to(exp.device)
            batch_y_mark = batch_y_mark.float().to(exp.device)

            layer_embeddings.clear()

            print(f"\nProcessing batch with shapes:")
            print(f"batch_x: {batch_x.shape}")
            print(f"batch_y: {batch_y.shape}")

            output = exp.model(batch_x, batch_x_mark, batch_y, batch_y_mark)
            print(f"Model output shape: {output.shape}")

            print(f"\nCaptured embeddings from {len(layer_embeddings)} layers:")
            for layer_name, embeddings in layer_embeddings.items():
                print(f"  {layer_name}: {embeddings.shape}")

    for handle in hook_handles:
        handle.remove()

    return layer_embeddings

def extract_final_embeddings(exp, layer_name):
    """
    Extract embeddings from a specific layer across all test data
    """
    test_data, test_loader = exp._get_data(flag='test')

    embeddings_list = []

    def embedding_hook(module, input, output):
        if isinstance(output, torch.Tensor):
            embeddings_list.append(output.detach().cpu().clone())

    hook_handle = None
    for name, module in exp.model.named_modules():
        if name == layer_name:
            hook_handle = module.register_forward_hook(embedding_hook)
            print(f"Hooked onto layer: {name}")
            break

    if not hook_handle:
        print(f"Layer {layer_name} not found!")
        return None

    exp.model.eval()
    all_embeddings = []

    with torch.no_grad():
        for i, (batch_x, batch_y, batch_x_mark, batch_y_mark) in enumerate(test_loader):
            batch_x = batch_x.float().to(exp.device)
            batch_y = batch_y.float().to(exp.device)
            batch_x_mark = batch_x_mark.float().to(exp.device)
            batch_y_mark = batch_y_mark.float().to(exp.device)

            embeddings_list.clear()

            _ = exp.model(batch_x, batch_x_mark, batch_y, batch_y_mark)

            if embeddings_list:
                all_embeddings.append(embeddings_list[0])

            if i % 10 == 0:
                print(f"Processed batch {i}/{len(test_loader)}")

    hook_handle.remove()

    if all_embeddings:
        final_embeddings = torch.cat(all_embeddings, dim=0).numpy()
        print(f"Final embeddings shape: {final_embeddings.shape}")
        return final_embeddings
    else:
        print("No embeddings captured!")
        return None

config = {
    'task_name': 'long_term_forecast',
    'is_training': 0,
    'model_id': 'test_btc_ohlcv_timesnet',
    'model': 'TimesNet',
    'data': 'custom',
    'root_path': './dataset/my_data/',
    'data_path': 'test_btc_ohlcv.csv',
    'features': 'MS',
    'target': 'return',
    'freq': 'h',
    'seasonal_patterns': 'Monthly',
    'inverse': 0,
    'seq_len': 48,
    'label_len': 24,
    'pred_len': 1,
    'e_layers': 1,
    'd_layers': 1,
    'factor': 3,
    'num_kernels': 6,
    'enc_in': 12,
    'dec_in': 12,
    'c_out': 1,
    'd_model': 64,
    'd_ff': 32,
    'top_k': 5,
    'des': 'QuickTest',
    'itr': 1,
    'batch_size': 32,
    'dropout': 0.1,
    'embed': 'timeF',
    'activation': 'gelu',
    'num_workers': 10,
    'patience': 3,
    'learning_rate': 0.0001,
    'use_gpu': True,
    'gpu_type': 'cuda',
    'use_multi_gpu': False,
    'gpu': 0
}

print("Step 1: Analyzing model structure...")
exp = debug_model_structure('auto', config)  #'auto' or checkpoint path

print("\nStep 2: Testing hooks on multiple layers...")
layer_embeddings = extract_embeddings_with_multiple_hooks(exp)

Step 1: Analyzing model structure...
Use GPU: cuda:0
=== TimesNet Model Structure ===
: Model
model: ModuleList
model.0: TimesBlock
model.0.conv: Sequential
model.0.conv.0: Inception_Block_V1
model.0.conv.0.kernels: ModuleList
model.0.conv.0.kernels.0: Conv2d
model.0.conv.0.kernels.1: Conv2d
model.0.conv.0.kernels.2: Conv2d
model.0.conv.0.kernels.3: Conv2d
model.0.conv.0.kernels.4: Conv2d
model.0.conv.0.kernels.5: Conv2d
model.0.conv.1: GELU
model.0.conv.2: Inception_Block_V1
model.0.conv.2.kernels: ModuleList
model.0.conv.2.kernels.0: Conv2d
model.0.conv.2.kernels.1: Conv2d
model.0.conv.2.kernels.2: Conv2d
model.0.conv.2.kernels.3: Conv2d
model.0.conv.2.kernels.4: Conv2d
model.0.conv.2.kernels.5: Conv2d
enc_embedding: DataEmbedding
enc_embedding.value_embedding: TokenEmbedding
enc_embedding.value_embedding.tokenConv: Conv1d
enc_embedding.position_embedding: PositionalEmbedding
enc_embedding.temporal_embedding: TimeFeatureEmbedding
enc_embedding.temporal_embedding.embed: Linear
enc_emb